In [0]:
#Import Google Drive functionality & mount Google Drive to access datasets
from google.colab import drive
#import
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.impute import KNNImputer
import seaborn as sns
import datetime
import time
import random
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Masking, LSTM, CuDNNLSTM, Dropout
from tensorflow.python.keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras import optimizers
drive.mount('/content/drive', force_remount=True)

In [0]:
#read in data

#### Data available at https://www.kaggle.com/c/ashrae-energy-prediction/data ####
meterReadings = pd.read_csv('/content/drive/My Drive/colab/data/ashrae-energy-prediction/train.csv')
buildingInfo = pd.read_csv('/content/drive/My Drive/colab/data/ashrae-energy-prediction/building_metadata.csv')
weatherReadings = pd.read_csv('/content/drive/My Drive/colab/data/ashrae-energy-prediction/weather_train.csv')
#### Data available at https://www.kaggle.com/c/ashrae-energy-prediction/data ####

#convert meter reading zero values to NaN
meterReadings.meter_reading.replace(0, np.nan, inplace=True)

#sanity check. Are any buildings built during/after the data collection timeframe?
newBuildings = buildingInfo.building_id.loc[buildingInfo.year_built >= 2016]
#remove those buildings from data. Can't provide accurate prediction if they weren't fully built
#or some data is from during construction, some after it was built. No data on day/month built to filter
meterReadings = meterReadings.loc[~meterReadings.building_id.isin(newBuildings)]

#convert timestamp values into datetime format for easier processing
meterReadings['timestamp'] = pd.to_datetime(meterReadings['timestamp'], format="%Y-%m-%d %H:%M:%S")
weatherReadings['timestamp'] = pd.to_datetime(weatherReadings['timestamp'], format="%Y-%m-%d %H:%M:%S")
meterReadings['timestamp'] = meterReadings['timestamp'].dt.to_pydatetime()
weatherReadings['timestamp'] = weatherReadings['timestamp'].dt.to_pydatetime()

#convert building at site 0 from kBtu to kWh
siteZero = buildingInfo['building_id'].loc[buildingInfo['site_id'] == 0].max()
meterReadings.loc[meterReadings['building_id'] <= siteZero, ['meter_reading']] *= 0.293071

#append the site_id of the building to each meter reading
meterReadings.insert(1, 'site_id', "")
for i in range (0,16):
    maxBuilding = buildingInfo.loc[buildingInfo.site_id == i].building_id.max()
    minBuilding = buildingInfo.loc[buildingInfo.site_id == i].building_id.min()
    meterReadings.site_id[meterReadings.loc[(meterReadings.building_id >= minBuilding) & (meterReadings.building_id <= maxBuilding)].index] = i

#convert 'primary_use' in buildingInfo to one hot encoding, for easier handling
primaryUseOneHot = pd.get_dummies(buildingInfo['primary_use'], prefix='use', dummy_na=True)
buildingInfo = pd.concat([buildingInfo, primaryUseOneHot], axis=1)

#reduce the memory usage for each dataset
def reduceDataMemUsage(data):
    #datatypes used by numpy. No int8 as it's the smallest data type, no need to check if data fits in smaller type
    dataTypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in data.columns:
        #get each column data type, check if in the list of possible types
        colDataType = data[col].dtypes
        if colDataType in dataTypes:
            colMinValue = data[col].min()
            colMaxValue = data[col].max()
            #check cols default datatype, see if the min/max values allows all data to fit in a smaller data type
            if str(colDataType)[:3] == 'int':
                #if min/max of col fits in range of smaller data type, convert whole col to that data type
                if colMinValue > np.iinfo(np.int8).min and colMaxValue < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif colMinValue > np.iinfo(np.int16).min and colMaxValue < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif colMinValue > np.iinfo(np.int32).min and colMaxValue < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif colMinValue > np.iinfo(np.int64).min and colMaxValue < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)
            else:
                if colMinValue > np.finfo(np.float16).min and colMaxValue < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif colMinValue > np.finfo(np.float32).min and colMaxValue < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)
    return data

#perform memory reduction function on each dataset
for x in [meterReadings, weatherReadings, buildingInfo]:
    x = reduceDataMemUsage(x)

#create electricReadings of just electricity readings
electricReadings = meterReadings[meterReadings['meter'] == 0]

#get all of the unique timestamps in the dataset
dates = electricReadings['timestamp'].unique()
electricTrain = pd.DataFrame()
electricTest = pd.DataFrame()

# FOR 'SOFT RESET', the initial data doesn't have to be read in again for train/test operations
electricTrain = pd.DataFrame()
electricTest = pd.DataFrame()

#split each 3 days of data into 2 days of training, 1 day of testing
for i in range(0,8784,72):
    electricTrain = electricTrain.append(electricReadings.loc[(electricReadings.timestamp >= dates[i]) & (electricReadings.timestamp <= dates[i+47])])
    electricTest = electricTest.append(electricReadings.loc[(electricReadings.timestamp >= dates[i+48]) & (electricReadings.timestamp <= dates[i+71])])

#drop all site 0 data from before 2016-05-21 00:00:00 - too complex to easily correct for missing data
electricTrain.drop(electricTrain.loc[(electricTrain.site_id == 0) & (electricTrain.timestamp <= '2016-05-21 00:00:00')].index, inplace=True)
electricTest.drop(electricTest.loc[(electricTest.site_id == 0) & (electricTest.timestamp <= '2016-05-21 00:00:00')].index, inplace=True)

def outlierRemoval(df,x,focus,start,end):
	'''Perform Outlier Removal using a moving median

	Parameters
	----------
	df : DataFrame 
		Dataframe to use
	x : String 
		Dataframe column to remove outliers from
	focus : String
		Dataframe column to use for grouping data (by building, site etc)
	start : int 
		Starting value for group (building_id of 0)
	end : int
		Ending value for group (building_id of 1449)

	Returns
	-------
	Filtered
		A DataFrame with outliers removed

	'''
	Filtered = pd.DataFrame()
	for i in range (start,end):
	    #get deep copy of data for each group
	    newList = df.loc[df[focus] == i].copy(deep=True)
	    #calculate median and standard deviation for each data window
	    newList['Rollingmedian_%s' %x] = newList[x].rolling(5).median()
	    newList['stdDev'] = newList[x].rolling(5).std()
	    #append filtered data to new dataframe (faster than modifying original dataframe)
	    Filtered = Filtered.append(newList)
	return(Filtered)
 
def fillMissingVals(df,x,focus,start,end):
	'''Perform imputation using Pandas' time interpolation

	Parameters
	----------
	df : DataFrame 
		Dataframe to use
	x : String 
		Dataframe column to impute
	focus : String
		Dataframe column to use for grouping data (by building, site etc)
	start : int 
		Starting value for group (building_id of 0)
	end : int
		Ending value for group (building_id of 1449)

	Returns
	-------
	imputed
		An imputed DataFrame

	'''
	imputed = pd.DataFrame()
	for i in range(start,end):
	    toInterpolate = df.loc[df[focus] == i].copy(deep=True)
	    toInterpolate = toInterpolate.set_index('timestamp')
	    toInterpolate[x] = toInterpolate[x].interpolate(method='time')
	    imputed = imputed.append(toInterpolate)
	imputed = imputed.reset_index()
	return(imputed)
 
def smoothVals(df,x,focus,start,end):
	'''Perform data smoothing using rolling mean

	Parameters
	----------
	df : DataFrame 
		Dataframe to use
	x : String 
		Dataframe column to smooth
	focus : String
		Dataframe column to use for grouping data (by building, site etc)
	start : int 
		Starting value for group (building_id of 0)
	end : int
		Ending value for group (building_id of 1449)

	Returns
	-------
	Filtered
		A DataFrame with additional row of smoothed data

	'''
	rolling = pd.DataFrame()
	for i in range(start,end):
	    #get deep copy of data for each group
	    vals = df.loc[df[focus] == i].copy(deep=True)
	    vals = vals.set_index('timestamp')
	    #calculate mean for each data window
	    vals['%sRollingAvg' %x] = vals[x].rolling(5, center=True, min_periods=1).mean()
	    rolling = rolling.append(vals)
	rolling = rolling.reset_index()
	return(rolling)
 

## Code adapted from 
## Brownlee, J. 2019. How to Convert a Time Series to a Supervised Learning Problem in Python. [Source code]
## Available at: https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
## [Accessed: 07 April 2020]
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


def evaluateModel():
    #predict values with test data
    predictedValues = model.predict(test_X)
    #turn 3D results array back into 2D table form
    test_X2 = test_X.reshape((test_X.shape[0], test_X.shape[1]*test_X.shape[2]))
    #invert scaling for forecasted labels
    invertedPredictedValues = np.concatenate((test_X2[:, -num_features:],predictedValues), axis=1)
    #replace masked values with NaN before inverting
    invertedPredictedValues[invertedPredictedValues == maskValue] = np.nan
    invertedPredictedValues = scaler.inverse_transform(invertedPredictedValues)
    invertedPredictedValues = invertedPredictedValues[:,-1]
    #reshape & invert scaling for the actual test labels
    test_y2 = test_y.reshape((len(test_y), 1))
    invertedActualValues = np.concatenate((test_X2[:, -num_features:], test_y2), axis=1)
    #replace masked values with NaN
    invertedPredictedValues[invertedActualValues[:,-1] == maskValue] = np.nan
    invertedActualValues[invertedActualValues == maskValue] = np.nan
    invertedActualValues = scaler.inverse_transform(invertedActualValues)
    invertedActualValues = invertedActualValues[:,-1]
    #remove NaN values so RMSE can be calculated. Should remove the same values for prediction and actual
    invertedPredictedValues = invertedPredictedValues[~np.isnan(invertedPredictedValues)]
    invertedActualValues = invertedActualValues[~np.isnan(invertedActualValues)]
    #calculate & return the root mean squared error between the prediction and actual values
    return np.sqrt(mean_squared_error(invertedActualValues, invertedPredictedValues))

#prepare site 2 data for machine learning
electricTrain = outlierRemoval(electricTrain,'meter_reading','building_id',156,290)
electricTest = outlierRemoval(electricTest,'meter_reading','building_id',156,290)

#interpolate missing weather values
weatherReadings = fillMissingVals(weatherReadings,'air_temperature','site_id',0,16)
weatherReadings = fillMissingVals(weatherReadings,'sea_level_pressure','site_id',0,16)
weatherReadings = fillMissingVals(weatherReadings,'wind_speed','site_id',0,16)

#remove outlier weather values
weatherReadings = outlierRemoval(weatherReadings,'air_temperature','site_id',0,16)
weatherReadings = outlierRemoval(weatherReadings,'sea_level_pressure','site_id',0,16)
weatherReadings = outlierRemoval(weatherReadings,'wind_speed','site_id',0,16)

In [0]:
buildingStart = 156
buildingEnd = 290
past_hours = 24
future_hours = 1
num_features = 4
num_observations = past_hours * num_features
maskValue = -1

electricTrainUseful = electricTrain[['building_id','site_id','Rollingmedian_meter_reading','timestamp']]
electricTestUseful = electricTest[['building_id','site_id','Rollingmedian_meter_reading','timestamp']]
#range of dates to reindex
idx = pd.date_range('2016-01-01 00:00:00', '2016-12-31 23:00:00', freq='H')
electricTrainReindexed = pd.DataFrame()
electricTestReindexed = pd.DataFrame()
#reindex every building's data to contain every date in the specified date range
for i in range(buildingStart, buildingEnd):
    #if training building has data
    if len(electricTrainUseful.loc[electricTrainUseful['building_id'] == i].values) !=0:
        electricTraining = electricTrainUseful.loc[electricTrainUseful.building_id == i].copy(deep=True)
        bid = electricTraining[:1].values[0,0]
        sid = electricTraining[:1].values[0,1]
        #reindex
        electricTraining = electricTraining.set_index('timestamp').reindex(idx).reset_index()
        electricTraining['site_id'] = sid
        electricTraining['building_id'] = bid
        electricTrainReindexed = electricTrainReindexed.append(electricTraining)
    #if testing building has data
    if len(electricTestUseful.loc[electricTestUseful['building_id'] == i].values) !=0:
        electricTesting2 = electricTestUseful.loc[electricTestUseful.building_id == i].copy(deep=True)
        bid = electricTesting2[:1].values[0,0]
        sid = electricTesting2[:1].values[0,1]
        #reindex
        electricTesting2 = electricTesting2.set_index('timestamp').reindex(idx).reset_index()
        electricTesting2['site_id'] = sid
        electricTesting2['building_id'] = bid
        electricTestReindexed = electricTestReindexed.append(electricTesting2)

#convert timedate64[ns] into hours elapsed since 2016-01-01T00:00:00, so it can go into the model as a float
electricTrainReindexed['hoursElapsed'] = (electricTrainReindexed['index'] - np.datetime64('2016-01-01T00:00:00'))/np.timedelta64(1, 'h')
electricTestReindexed['hoursElapsed'] = (electricTestReindexed['index'] - np.datetime64('2016-01-01T00:00:00'))/np.timedelta64(1, 'h')
#merge data from other datasets
electricTrainReindexed = electricTrainReindexed.merge(weatherReadings[['site_id', 'timestamp', 'Rollingmedian_air_temperature', 'Rollingmedian_sea_level_pressure']], how='left', left_on=['site_id', 'index'], right_on=['site_id', 'timestamp'])
electricTrainReindexed = electricTrainReindexed.merge(buildingInfo[['building_id', 'site_id', 'square_feet']], how='left', on=['building_id', 'site_id'])
electricTestReindexed = electricTestReindexed.merge(weatherReadings[['site_id', 'timestamp', 'Rollingmedian_air_temperature', 'Rollingmedian_sea_level_pressure']], how='left', left_on=['site_id', 'index'], right_on=['site_id', 'timestamp'])
electricTestReindexed = electricTestReindexed.merge(buildingInfo[['building_id', 'site_id', 'square_feet']], how='left', on=['building_id', 'site_id'])
#sort so all building data at given time are together
electricTrainReindexed = electricTrainReindexed.sort_values(by=['index', 'building_id'])
electricTestReindexed = electricTestReindexed.sort_values(by=['index', 'building_id'])
#drop values that aren't floats, such as the timestamps which have been converted to hoursElapsed
electricTrainReindexed = electricTrainReindexed[['hoursElapsed', 'square_feet', 'Rollingmedian_air_temperature', 'Rollingmedian_sea_level_pressure', 'Rollingmedian_meter_reading']]
electricTestReindexed = electricTestReindexed[['hoursElapsed', 'square_feet', 'Rollingmedian_air_temperature', 'Rollingmedian_sea_level_pressure', 'Rollingmedian_meter_reading']]

#set a whole row's data to NaN where its meter reading is NaN. 
#Will be replaced and ignored in the model
electricTrainReindexed.loc[electricTrainReindexed.Rollingmedian_meter_reading.isnull()] = np.nan
electricTestReindexed.loc[electricTestReindexed.Rollingmedian_meter_reading.isnull()] = np.nan

#scale the values. Removes all column/dataframe info, only arrays from now on
#fit scaler on the training data only, transform both with the training scaler parameters
scaler = MinMaxScaler()
electricTrainReindexedScaled = scaler.fit_transform(electricTrainReindexed)
electricTestReindexedScaled = scaler.transform(electricTestReindexed)

#replace all NaN values with mask value to be dropped in model
#must happen after scaling otherwise mask value will be scaled with everything else
electricTrainReindexedScaled = np.nan_to_num(electricTrainReindexedScaled, nan=maskValue)
electricTestReindexedScaled = np.nan_to_num(electricTestReindexedScaled, nan=maskValue)

#shift values to create x hours worth of data
trainData = series_to_supervised(electricTrainReindexedScaled, past_hours, future_hours).values
testData = series_to_supervised(electricTestReindexedScaled, past_hours, future_hours).values

#split data into features & label (meter reading)
colsToDelete = []
futureColsToDelete = []
#generate list of columns which contain each label for previous timesteps we don't want
#want to delete the label for the first column (0 indexed so num_features addresses the label after the last feature)
#and then up to the last past hour label
for i in range(num_features,past_hours*(num_features+1),num_features+1):
    colsToDelete.append(i)
#list of columns for the future timesteps we don't want (all of the future's variables. Want only the labels for every future hour)
for i in range(past_hours*(num_features+1), trainData.shape[1],num_features+1):
    futureColsToDelete.extend(range(i,i+num_features))

#delete all labels for past data & delete all observations for future data, split into _X and _y
trainData = np.delete(trainData,futureColsToDelete, axis=1)
trainData = np.delete(trainData,colsToDelete, axis=1)
testData = np.delete(testData,futureColsToDelete, axis=1)
testData = np.delete(testData,colsToDelete, axis=1)
train_X, train_y = trainData[:,:-future_hours], trainData[:,-1]
test_X, test_y = testData[:,:-future_hours], testData[:,-1]
#FUTURE WORK. Set train_y/test_y to a series of labels for multiple future timesteps. Potentially sum together future labels into one
#train_X, train_y = trainData[:,:-future_hours], trainData[:,num_observations:]
#test_X, test_y = testData[:,:-future_hours], testData[:,num_observations:]

#reshape features into a 3D array
train_X = train_X.reshape(train_X.shape[0], 1, num_features*past_hours)
test_X = test_X.reshape(test_X.shape[0], 1, num_features*past_hours)

In [0]:
def trainModels(numNodes, numEpochs, batchSize, optimiserToUse, activationChosen, maskingEnabled, dropoutEnabled, dropoutValue, useGPU, model):
    #design network
    if(maskingEnabled):
        model.add(Masking(mask_value=maskValue ,input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(LSTM(numNodes))
    else:
        if(useGPU):
            model.add(CuDNNLSTM(numNodes, input_shape=(train_X.shape[1], train_X.shape[2])))
        else:
            model.add(LSTM(numNodes, input_shape=(train_X.shape[1], train_X.shape[2])))
    if(dropoutEnabled):
        model.add(Dropout(dropoutValue))
    model.add(Dense(1, activation=activationChosen))
    model.compile(loss='mse', optimizer=optimiserToUse)
    #fit network
    history = model.fit(train_X, train_y, epochs=numEpochs, batch_size=batchSize, validation_split=0.33, verbose=0, shuffle=False)
    return history

plt.figure(figsize=(12,6))
#used to conveniently replace words in plot labels & result outputs
toModify = 'epochs'

for n in [16]:
    print('Running node_size %s'%n)
    for i in [10,20,30,40,50,60,70,80,90,100]:
        runValues = []
        for j in range(1,4):
            #run each experiment 3 times for an avg
            model = Sequential()
            optimiserToUse = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
            history = trainModels(n, i, past_hours*(buildingEnd-buildingStart), optimiserToUse, 'relu', False, True, 0.2, False, model)
            result = evaluateModel()
            runValues.append(result)
            #print RMSE result in LaTeX Tabular friendly way
            print('%.4f'%(result),sep='', end='&')
            if(j==2):
                #plot the last run only
                plt.plot(history.history['loss'], label='Train %d %s'%(i, toModify), linestyle='-')
                plt.plot(history.history['val_loss'], label='Val %d %s'%(i, toModify), linestyle='--')
            tf.keras.backend.clear_session()
        print('Avg for %s:%d RMSE:%.4f'%(toModify, i, np.average(runValues)))

    plt.title('How the varying of %s affects training & validation loss'%toModify)
    plt.xlabel('Number of %s'%toModify)
    plt.ylabel('Loss')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.09),
            fancybox=True, shadow=False, ncol=5)
    #plt.savefig('/content/drive/My Drive/colab/varyingEpochsCPU_%sNodesReluDropout0_2v2.png'%n, bbox_inches='tight')
    #plt.clf()

In [0]:
# Predict test set on the model, invert results

#predict values with test data
predictedValues = model.predict(test_X)

testDataReindexed = np.nan_to_num(electricTestReindexed, nan=maskValue)
testDataShifted = series_to_supervised(testDataReindexed, past_hours, future_hours).values
#filter to only the values for each timestep
testDataValueSequence = testDataShifted[:,colsToDelete]
#turn 3D results array back into 2D table form
test_X2 = test_X.reshape((test_X.shape[0], test_X.shape[1]*test_X.shape[2]))
#invert scaling for forecasted labels
invertedPredictedValues = np.concatenate((test_X2[:, -num_features:],predictedValues), axis=1)
#replace masked values with NaN before inverting
invertedPredictedValues[invertedPredictedValues == maskValue] = np.nan
invertedPredictedValues = scaler.inverse_transform(invertedPredictedValues)
invertedPredictedValues = invertedPredictedValues[:,-1]
#reshape & invert scaling for the actual test labels
test_y2 = test_y.reshape((len(test_y), 1))
invertedActualValues = np.concatenate((test_X2[:, -num_features:], test_y2), axis=1)
#replace masked values with NaN
invertedPredictedValues[invertedActualValues[:,-1] == maskValue] = np.nan
invertedActualValues[invertedActualValues == maskValue] = np.nan
invertedActualValues = scaler.inverse_transform(invertedActualValues)
invertedActualValues = invertedActualValues[:,-1]

In [0]:
# plot random sequence of readings. May need to run several times to get 
# a sequence plot that isn't just the masking value
randomValue = random.randint(0, invertedActualValues.shape[0]-1)
sequenceToPlot = testDataValueSequence[randomValue,:]

plt.figure(figsize=(12,6))
plt.plot(sequenceToPlot, label='Prior sequence meter readings')
plt.plot([24],invertedPredictedValues[randomValue], marker='x', label='Predicted meter reading')
plt.plot([24],invertedActualValues[randomValue], marker='o', label='Actual meter reading')

futurePredictions = []
futureActuals = []
for i in range(1,24):
    plt.plot([24+i],invertedPredictedValues[randomValue+i], marker='x', label='Predicted+%s meter reading'%i)
    plt.plot([24+i],invertedActualValues[randomValue+i], marker='o', label='Actual+%s meter reading'%i)

plt.title('Predicting the next 24 meter readings of test data sequence %d' %randomValue)
plt.xlabel('Point in sequence')
plt.ylabel('Meter Reading (kWh)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.09),
        fancybox=True, shadow=False, ncol=5)

#plt.savefig('/content/drive/My Drive/colab/sequenceFuture24PredictionGraph_%s.png'%randomValue, bbox_inches='tight')

Below are some snippets of code not needed to run the current solution, but were used at one point

In [0]:
#create heatmap of absolute Pearson correlation between all dataset columns
merged = pd.merge(electricTrain, weatherReadings)
merged = pd.merge(merged, buildingInfo)
plt.figure(figsize=(10,8))
testMatrix = merged.corr().abs()
sns.heatmap(testMatrix)
plt.title('Absolute Pearson correlation heatmap between the \n electricTrain, weatherReadings and buildingInfo dataframes')
#plt.savefig('/content/drive/My Drive/colab/corrHeatmap.png', bbox_inches='tight')

In [0]:
# Predict test set on the model, print RMSE & random sample of predictions vs actual values

#predict values with test data
predictedValues = model.predict(test_X)
#turn 3D results array back into 2D table form
test_X2 = test_X.reshape((test_X.shape[0], test_X.shape[1]*test_X.shape[2]))
#invert scaling for forecasted labels
invertedPredictedValues = np.concatenate((test_X2[:, -num_features:],predictedValues), axis=1)
#replace masked values with NaN before inverting
invertedPredictedValues[invertedPredictedValues == maskValue] = np.nan
invertedPredictedValues = scaler.inverse_transform(invertedPredictedValues)
invertedPredictedValues = invertedPredictedValues[:,-1]
#reshape & invert scaling for the actual test labels
test_y2 = test_y.reshape((len(test_y), 1))
invertedActualValues = np.concatenate((test_X2[:, -num_features:], test_y2), axis=1)
#replace masked values with NaN
invertedPredictedValues[invertedActualValues[:,-1] == maskValue] = np.nan
invertedActualValues[invertedActualValues == maskValue] = np.nan
invertedActualValues = scaler.inverse_transform(invertedActualValues)
invertedActualValues = invertedActualValues[:,-1]
#remove NaN values so RMSE can be calculated. Should remove the same values for prediction and actual
invertedPredictedValues = invertedPredictedValues[~np.isnan(invertedPredictedValues)]
invertedActualValues = invertedActualValues[~np.isnan(invertedActualValues)]
#calculate & return the root mean squared error between the prediction and actual values
rmse = np.sqrt(mean_squared_error(invertedActualValues, invertedPredictedValues))
print('Test RMSE: %.5f' % rmse)
#print random sample of prediction and actual values
print('\nRandom sample of predictions vs actual values:')
for i in range(0,10):
    randomValue = random.randint(0, invertedActualValues.shape[0]-1)
    print('value %7d, predicted: %.5f, actual: %.5f, difference: %.5f (%.2f%%)' % (randomValue, invertedPredictedValues[randomValue], 
                                                                          invertedActualValues[randomValue], 
                                                                          (invertedPredictedValues[randomValue] - invertedActualValues[randomValue]),
                                                                          ((invertedPredictedValues[randomValue] - invertedActualValues[randomValue])/abs(invertedActualValues[randomValue])*100)))

In [0]:
#Visualise rolling meter readings
#electricTrain = electricTrain.reset_index()
IdToProcess = 231
data = electricTrainTest.loc[electricTrainTest.building_id == 231]
print(data)
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(data['index'], data.Rollingmedian)
ax.set_xlabel('Date')
ax.set_ylabel('Electricity Usage (kWh)')
ax.set_title('Building ID: %s \nRolling median meter readings (window size 5)' %IdToProcess)
#plt.savefig('/content/drive/My Drive/colab/graphics/trainBuilding' + str(IdToProcess) + '_rollingmedian_5.png', bbox_inches='tight')
plt.show()

In [0]:
#calculate how much data/percent of data is missing from each dataset, printed into LaTeX tables
def calcMissing:
    totalMissing = weatherReadings.isnull().sum().sort_values(ascending=False)
    percentMissing = (weatherReadings.isnull().sum()/len(weatherReadings.index)*100).sort_values(ascending=False)
    missingweatherReadingsData = pd.concat([totalMissing, percentMissing], keys=['Total Values Missing', 'Percent of Values Missing'], axis=1)
    #print(missingweatherReadingsData.to_latex(index=True))

    totalMissing = meterReadings.isnull().sum().sort_values(ascending=False)
    percentMissing = (meterReadings.isnull().sum()/len(meterReadings.index)*100).sort_values(ascending=False)
    missingmeterReadingsData = pd.concat([totalMissing, percentMissing], keys=['Total Values Missing', 'Percent of Values Missing'], axis=1)
    #print(missingmeterReadingsData.to_latex(index=True))

    totalMissing = buildingInfo.isnull().sum().sort_values(ascending=False)
    percentMissing = (buildingInfo.isnull().sum()/len(buildingInfo.index)*100).sort_values(ascending=False)
    missingBuildingInfoData = pd.concat([totalMissing, percentMissing], keys=['Total Values Missing', 'Percent of Values Missing'], axis=1)
    #print(missingBuildingInfoData.to_latex(index=True))

In [0]:
#Visualise how much data exists, and whether it is NaN, or zero
meterReadings = meterReadings.set_index(['timestamp'])
f, a=plt.subplots(1, 4, figsize=(23, 30))
for meterType in np.arange(4): #for each type of meter
    converted = meterReadings[meterReadings['meter'] == meterType].copy().reset_index() #copy only the same meter type values to df, and reset the index to default
    converted['timestamp'] = (pd.to_timedelta(converted['timestamp']).dt.total_seconds() / 3600).astype(int) #convert the timestamp col values to hours since start of timestamp (1677-09-21)
    converted['timestamp'] -= converted['timestamp'].min() #subtract the min time supported by timestamp (1677-09-21) to get the time since 1st value
    missingValues = np.empty((1449, converted['timestamp'].max() + 1)) #create new empty dataframe of size 1449 x max number in timestamp (each building's entire readings per row)
    missingValues.fill(np.nan) #fill dataframe with NaN values
    for meterReading in converted.values:
        missingValues[int(meterReading[1]), int(meterReading[0])] = 0 if meterReading[4] == 0 else 1 #set dataframe at row building No, column hour elapsed with 0 if value is 0, 1 if non-0 value exists
    a[meterType].set_title(f'meter {meterType:d}')
    sns.heatmap(missingValues, cmap='Paired', ax=a[meterType], cbar=False)
meterReadings = meterReadings.reset_index()
#plt.savefig('/content/drive/My Drive/colab/meterReadingsUnprocessed.png', bbox_inches='tight')

In [0]:
#Visualise how much electricity data exists, and whether it is NaN, or zero
#electricReadings = electricReadings.set_index(['timestamp'])
converted = electricTrain.copy() #copy only the same meter type values to df, and reset the index to default
converted['timestamp'] = (pd.to_timedelta(converted['timestamp']).dt.total_seconds() / 3600).astype(int) #convert the timestamp col values to hours since start of timestamp (1677-09-21)
converted['timestamp'] -= converted['timestamp'].min() #subtract the min time supported by timestamp (1677-09-21) to get the time since 1st value
missingValues = np.empty((1449, converted['timestamp'].max() + 1)) #create new empty dataframe of size 1449 x max number in timestamp (each building's entire readings per row)
missingValues.fill(np.nan) #fill dataframe with NaN values
for electricReading in converted.values:
    missingValues[int(electricReading[0]), int(electricReading[3])] = 0 if electricReading[4] == 0 else 1 #set dataframe at row building No, column hour elapsed with 0 if value is 0, 1 if non-0 value exists
sns.set(rc={'figure.figsize':(20, 30)})
plot = sns.heatmap(missingValues, cmap='Paired', cbar=False)
plt.title('Electricity training data visualisation', loc='center')
plt.xlabel('Hours from start of data')
plt.ylabel('Building ID')
#electricReadings = electricReadings.reset_index() #reset index after done, to allow other functions to work on the original dataset shape
#plt.savefig('/content/drive/My Drive/colab/electricityTrainUnprocessed.png', bbox_inches='tight')

In [0]:
## Code adapted from
## Hernández, J. 2019. ASHRAE - Outliers. [Source code]
## Available at: https://www.kaggle.com/juanmah/ashrae-outliers/notebook
## [Accessed: 24 February 2020]

electricTrainDaily = electricTrain
#group electric readings by date and then building ID. Sum all columns (though only interested in meter readings)
electricTrainDaily = electricTrainDaily.groupby([electricTrain.timestamp.dt.date, 'building_id']).sum()
#combine the hourly measurements within each date, providing the sum, mean, which index has max value & max value for each
electricTrainDailyAgg = electricTrainDaily.groupby(['timestamp']).agg(['sum', 'mean', 'idxmax', 'max'])
#reduce dimensions of dataframe, easier to access values
level_0 = electricTrainDailyAgg.columns.droplevel(0)
level_1 = electricTrainDailyAgg.columns.droplevel(1)
#change namings of headings
level_0 = ['' if x == '' else '-' + x for x in level_0]
electricTrainDailyAgg.columns = level_1 + level_0
electricTrainDailyAgg.rename_axis(None, axis=1)
#create list of which building_id uses maximum electricity for each date
dailyMaxElectricConsumers = [building[1] for building in electricTrainDailyAgg['meter_reading-idxmax']]
#count no. times each building_id appears in list, which building has most days of maximum consumption
from collections import Counter
print('Building ID: No. times max consumer:',Counter(dailyMaxElectricConsumers))
#print('Building ID: No. times max consumer:',Counter(dailyMaxElectricConsumers).to_latex(index=True))
#find out what type of buildings these max consumers are
print(buildingInfo.primary_use.loc[buildingInfo.building_id.isin(dailyMaxElectricConsumers)])

In [0]:
# Robust Z Score method. Too slow in current implementation

def getRobustZScore(x):
    #get the median of the sorted values. Numpy automatically sorts values
    windowMedian = np.median(x)
    #calculate Median Absolute Deviation
    medianAbsDev = np.median([np.abs(x - windowMedian) for windowSortedVal in x])
    #calculate z score
    robustZScore = 0.6745 * np.sum([x - windowMedian for windowSortedVal in x]) / medianAbsDev
    return np.abs(robustZScore)

for i in range(1,10):
    buildingList = electricTrain.loc[electricTrain.building_id == i].copy()
    buildingList['rZScore'] = buildingList.meter_reading.rolling(5).apply(getRobustZScore, raw=True)
#electricTrain.drop(buildingList.loc[buildingList.rZScore >= 3.5].index, inplace=True)

In [0]:
# Get the missing values
electricReadings.reset_index()
totalMissing = electricReadings.isnull().sum().sort_values(ascending=False)
percentMissing = (electricReadings.isnull().sum()/len(electricReadings.index)*100).sort_values(ascending=False)
missingelectricReadingsData = pd.concat([totalMissing, percentMissing], keys=['Total Values Missing', 'Percent of Values Missing'], axis=1)
print(missingelectricReadingsData.to_latex(index=True))